# QURI Parts入門 - 量子回路と量子状態の基礎

このノートブックでは、QURI Partsを使った量子計算の基本を学びます。

QURI Partsは、量子アルゴリズムの研究・開発のための統合的なPythonライブラリです。

## 1. 環境の確認とインポート

### Google Colabをお使いの場合
最初に以下のセルを実行してQURI Partsをインストールしてください：

In [ ]:
# Google Colab環境でQURI Partsをインストール
# MyBinder環境では既にインストール済みなので、このセルはスキップできます
try:
    import google.colab
    !python -m pip install -q quri-parts[qulacs,openfermion]==0.22.0 matplotlib
    print("QURI Partsのインストールが完了しました！")
except ImportError:
    print("MyBinder環境で実行中です。QURI Partsは既にインストールされています。")

In [ ]:
# 必要なライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt

# QURI Partsのコアモジュール
from quri_parts.core.state import quantum_state, ComputationalBasisState, GeneralCircuitQuantumState
from quri_parts.core.circuit import QuantumCircuit, H, X, Y, Z, CNOT, RX, RY, RZ
from quri_parts.core.measurement import bitwise_commuting_pauli_measurement
from quri_parts.core.operator import Operator, pauli_label, zero

print("QURI Partsが正常にインポートされました！")

## 2. 基本的な量子回路の作成

まず、簡単な量子回路を作成してみましょう。

In [ ]:
# 2量子ビットの回路を作成
circuit = QuantumCircuit(2)

# ゲートを追加
circuit.add_H_gate(0)  # 0番目の量子ビットにHadamardゲート
circuit.add_CNOT_gate(0, 1)  # CNOTゲート（制御: 0, 標的: 1）

print("作成した量子回路:")
print(circuit)
print(f"\n回路の深さ: {circuit.depth}")
print(f"ゲート数: {len(circuit.gates)}")

## 3. 量子状態の作成と操作

QURI Partsでは、様々な方法で量子状態を作成できます。

In [ ]:
# 計算基底状態 |00⟩ を作成
zero_state = ComputationalBasisState(2, bits=0b00)

# 回路を適用して新しい状態を作成（Bell状態）
bell_state = GeneralCircuitQuantumState(2, circuit)

print("Bell状態が作成されました")
print(f"量子ビット数: {bell_state.qubit_count}")
print(f"適用された回路の深さ: {bell_state.circuit.depth}")

## 4. 量子状態のシミュレーション

Qulacsバックエンドを使って状態ベクトルを計算します。

In [ ]:
# Qulacsシミュレータを使用
from quri_parts.qulacs.simulator import evaluate_state_to_vector

# Bell状態の状態ベクトルを計算
state_vector = evaluate_state_to_vector(bell_state)

print("Bell状態の状態ベクトル:")
for i, amplitude in enumerate(state_vector.vector):
    if abs(amplitude) > 1e-10:
        print(f"|{i:02b}⟩: {amplitude:.3f}")

# 確率分布を計算
probabilities = np.abs(state_vector.vector)**2
print("\n測定確率:")
for i, prob in enumerate(probabilities):
    if prob > 1e-10:
        print(f"|{i:02b}⟩: {prob:.3f}")

## 5. パラメトリック量子回路

変分量子アルゴリズムで使用されるパラメトリック回路を作成します。

In [ ]:
from quri_parts.core.circuit import ParametricQuantumCircuit

# パラメトリック回路を作成
param_circuit = ParametricQuantumCircuit(2)

# パラメータ付きゲートを追加
param_circuit.add_ParametricRY_gate(0)  # θ0
param_circuit.add_ParametricRY_gate(1)  # θ1
param_circuit.add_CNOT_gate(0, 1)
param_circuit.add_ParametricRZ_gate(0)  # θ2
param_circuit.add_ParametricRZ_gate(1)  # θ3

print(f"パラメータ数: {param_circuit.parameter_count}")
print("\nパラメトリック回路:")
print(param_circuit)

In [ ]:
# パラメータに具体的な値を代入
params = [np.pi/4, np.pi/3, np.pi/6, np.pi/2]
concrete_circuit = param_circuit.bind_parameters(params)

print("パラメータを代入した回路:")
print(concrete_circuit)

# この回路で状態を作成
param_state = GeneralCircuitQuantumState(2, concrete_circuit)
param_vector = evaluate_state_to_vector(param_state)

print("\n結果の状態ベクトル:")
for i, amplitude in enumerate(param_vector.vector):
    if abs(amplitude) > 1e-10:
        print(f"|{i:02b}⟩: {amplitude:.3f}")

## 6. 観測量と期待値

量子状態に対する観測量の期待値を計算します。

In [ ]:
# Pauli Z演算子を定義
z0 = Operator({pauli_label("Z0"): 1.0})  # Z on qubit 0
z1 = Operator({pauli_label("Z1"): 1.0})  # Z on qubit 1
zz = Operator({pauli_label("Z0 Z1"): 1.0})  # Z⊗Z

# 期待値を計算
from quri_parts.qulacs.estimator import create_qulacs_vector_estimator

estimator = create_qulacs_vector_estimator()

# Bell状態での期待値
exp_z0 = estimator(z0, bell_state)
exp_z1 = estimator(z1, bell_state)
exp_zz = estimator(zz, bell_state)

print("Bell状態での期待値:")
print(f"<Z0> = {exp_z0.value:.3f}")
print(f"<Z1> = {exp_z1.value:.3f}")
print(f"<Z0⊗Z1> = {exp_zz.value:.3f}")

## 7. 測定シミュレーション

量子状態の測定をシミュレートします。

In [ ]:
from quri_parts.qulacs.sampler import create_qulacs_vector_sampler

# サンプラーを作成
sampler = create_qulacs_vector_sampler()

# 1000回測定（回路を渡す）
n_shots = 1000
counts = sampler(circuit, shots=n_shots)  # bell_stateではなくcircuitを渡す

print(f"測定結果 ({n_shots}回):")
for bits, count in counts.items():
    print(f"|{bits:02b}⟩: {count}回 ({count/n_shots:.1%})")

# 測定結果を可視化
basis_labels = [f"|{i:02b}⟩" for i in range(4)]
measured_probs = [counts.get(i, 0) / n_shots for i in range(4)]
theoretical_probs = probabilities

x = np.arange(len(basis_labels))
width = 0.35

fig, ax = plt.subplots(figsize=(8, 6))
bars1 = ax.bar(x - width/2, theoretical_probs, width, label='理論値', alpha=0.8)
bars2 = ax.bar(x + width/2, measured_probs, width, label='測定値', alpha=0.8)

ax.set_xlabel('基底状態')
ax.set_ylabel('確率')
ax.set_title('Bell状態の測定結果')
ax.set_xticks(x)
ax.set_xticklabels(basis_labels)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## まとめ

このノートブックでは、QURI Partsの基本的な使い方を学びました：

1. **量子回路の作成**: `QuantumCircuit`クラスを使った回路構築
2. **量子状態の定義**: `ComputationalBasisState`と`GeneralCircuitQuantumState`
3. **状態ベクトルシミュレーション**: Qulacsバックエンドによる計算
4. **パラメトリック回路**: 変分量子アルゴリズムの基礎
5. **観測量と期待値**: Pauli演算子の期待値計算
6. **測定シミュレーション**: 確率的な測定結果の生成

QURI Partsは、これらの基本機能に加えて、量子化学計算、量子機械学習、エラー緩和など、高度な量子アルゴリズムの実装もサポートしています。